In [ ]:
### only colab
# ! git clone 'https://github.com/kangjun205/Dacon_AuthorClassification.git'
# %cd Dacon_AuthorClassification

# Data Reading

In [ ]:
import numpy as np
import pandas as pd
import wandb
from wandb.apis import public
import torch
from transformers import BertTokenizer

# ## only local
# sys.path.append('..')

from utils.clean import clean_texts

from src.model import BertForMultiLabelClassification

In [ ]:
test = pd.read_csv('test_x.csv') ## text for test
sample_submission = pd.read_csv('sample_submission.csv') ## submission file form
test.head()

# Model Loading

In [ ]:
## model hyperparameter loading
api = public.Api()
run_path = 'kangjun205/Dacon_AuthorClassification/run-id'  # run-id 수정
run = api.run(run_path)
params = run.config

## model state loading
model_file = run.file('bert_0412.pt').download(replace = True).name
model = BertForMultiLabelClassification(params['NUM_LABELS'], params['NUM_HIDDEN'])
model.load_state_dict(torch.load(model_file))

# Data PreProcessing

In [ ]:
## cleaning
test_data = clean_texts(test['text'])

## tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') ## tokenizer 호출
encoding = tokenizer.encode_plus(
    test_data,
    add_special_tokens=True,
    max_length=params['MAX_LEN'],
    return_token_type_ids=False,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)
input_ids = encoding['input_ids'].flatten()
attention_mask = encoding['attention_mask'].flatten()

# Prediction & Submission

In [ ]:
## device 설정
model.to('cuda' if torch.cuda.is_available() else 'cpu')
input_ids.to('cuda' if torch.cuda.is_available() else 'cpu')
attention_mask.to('cuda' if torch.cuda.is_available() else 'cpu')

## prediction
model.eval()
with torh.no_grad() :
    predicted_values = model(input_ids, attention_mask).to('cpu')
    predicted_values = pd.DataFrame(predicted_values.detach().numpy())

## submission file generation
sample_submission[['0', '1', '2', '3', '4']] = predicted_values
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')